Создадим новую БД с двумя таблицами — **video_products** и **original_titles**. 

В таблице **video_products** создадим три поля:

* `id` — primary key;

* `title` — название произведения (обязательное поле);

* `original_title_id` — foreign key; через это поле таблица **video_products** будет связана с таблицей **original_titles**. Это поле тоже сделаем обязательным: будем считать, что у любого произведения есть оригинальное название.

В таблице original_titles будет два поля:


* `id` — primary key;

* `title` — оригинальное название произведения (обязательное поле);

![alt text](https://pictures.s3.yandex.net/resources/S02_225_1685551181.png)

In [1]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS original_titles(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    original_title_id INTEGER NOT NULL UNIQUE,
    FOREIGN KEY(original_title_id) REFERENCES original_titles(id)
);
''')

con.close()

Вот как создаётся поле для связи с другой таблицей:  

1. В таблице **video_products** создаётся поле `original_title_id` типа `INTEGER` (только числа).

2. `UNIQUE` объявляет это поле уникальным в пределах таблицы: в ячейках этой колонки не может быть двух одинаковых значений — два разных произведения не должны ссылаться на одно и то же оригинальное название.

3. `NOT NULL` — поле не должно быть пустым, при добавлении новой записи это поле обязательно нужно заполнять.

4. `FOREIGN KEY(original_title_id)` — объявляет поле `original_title_id` внешним ключом.

5. Ключевое слово `REFERENCES` указывает, с каким полем какой таблицы связан `FOREIGN KEY`. В нашем случае это поле `id` в таблице **original_titles**.

Заполним таблицы. Для этого сопоставим русские названия произведений с оригинальными:

![alt text](https://pictures.s3.yandex.net/resources/S2_155_1682533726.png)

В таблице **original_titles** названия произведений могут идти в другом порядке, чем в таблице **video_products**, и храниться под другими id. При заполнении таблицы **video_products** важно указать правильный `original_title_id`, чтобы ссылка вела на нужную запись в таблице **original_titles**. 

In [2]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

original_titles = [
    (1, 'Last Action Hero'),
    (2, 'Murder, She Wrote'),
    (3, 'Looney Tunes'),
    (4, 'Il Buono, il brutto, il cattivo'),
    (5, 'Who Framed Roger Rabbit'),
    (6, 'Merrie Melodies'),
    (7, 'Mrs. \'Arris Goes to Paris')
]

video_products = [
    (1, 'Безумные мелодии Луни Тюнз', 3),
    (2, 'Весёлые мелодии', 6),
    (3, 'Кто подставил кролика Роджера', 5),
    (4, 'Хороший, плохой, злой', 4),
    (5, 'Последний киногерой', 1),
    (6, 'Она написала убийство', 2),
    (7, 'Миссис Харрис едет в Париж', 7)
]

cur.executemany('INSERT INTO original_titles VALUES(?, ?);', original_titles)
cur.executemany('INSERT INTO video_products VALUES(?, ?, ?);', video_products)

con.commit()
con.close()

В результате выполнения этого кода таблицы, связанные по типу «один к одному», будут наполнены данными.

***
## Получение информации из связанных таблиц

Выборку из таблиц, связанных 1:1, можно получить, например, таким запросом: 

```py
...

results = cur.execute('''
-- Вернуть поле title из таблицы video_products и поле title из original_titles
SELECT video_products.title,
       original_titles.title
-- ...из двух таблиц
FROM video_products,
     original_titles
-- Выводить только те значения полей, для которых верно условие
WHERE video_products.original_title_id = original_titles.id;
''')

for result in results:
    print(result)

...
```

Результат:

```bash
('Безумные мелодии Луни Тюнз', 'Looney Tunes')
('Весёлые мелодии', 'Merrie Melodies')
('Кто подставил кролика Роджера', 'Who Framed Roger Rabbit')
('Хороший, плохой, злой', 'Il Buono, il brutto, il cattivo')
('Последний киногерой', 'Last Action Hero')
('Она написала убийство', 'Murder, She Wrote')
('Миссис Харрис едет в Париж', "Mrs. 'Arris Goes to Paris") 
```

Вот дословный перевод запроса: «из таблиц **video_products** и **original_titles** верни столбцы `video_products.title` и `original_titles.title` и покажи пары полей, для которых выполняется условие `video_products.original_title_id = original_titles.id`».

При выборке из нескольких таблиц нужно указывать не только имя поля, но и имя таблицы, чтобы СУБД точно знала, о каком поле идёт речь.  

Длинным названиям столбцов лучше давать короткие псевдонимы с помощью ключевого слова `AS`:

In [5]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

results = cur.execute('''
SELECT video_products.title AS translation,
       original_titles.title AS original
FROM video_products,
     original_titles
WHERE 
    video_products.original_title_id = original_titles.id
  -- Для интереса добавим условие
  AND
    original LIKE 'M%';
''')

for result in results:
    print(result)

con.commit()
con.close()

('Весёлые мелодии', 'Merrie Melodies')
('Она написала убийство', 'Murder, She Wrote')
('Миссис Харрис едет в Париж', "Mrs. 'Arris Goes to Paris")


***
```sql
original LIKE 'M%'
```

Эта запись используется в языке запросов для работы с базами данных, например, в SQL. Она помогает найти данные, которые начинаются с определённой буквы или набора символов.

В данном случае `original LIKE 'M%'` означает:

* `original` — это название столбца в таблице базы данных, где хранятся текстовые данные.

* `LIKE` — это оператор, который позволяет искать данные, соответствующие определённому шаблону.

* `'M%'` — это шаблон, который говорит: «найди все значения, которые начинаются с буквы M и могут продолжаться любыми другими символами».

***
`cur.executescript` — это команда, которая позволяет выполнить сразу несколько SQL-запросов (команд для работы с базой данных) подряд, не делая каждый из них по отдельности.

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')

cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS ice_cream(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    wrapper_id INTEGER,
    FOREIGN KEY(wrapper_id) REFERENCES wrappers(id)
);
                  
CREATE TABLE IF NOT EXISTS wrappers(
    title TEXT NOT NULL,
    id INTEGER PRIMARY KEY
);
''')
con.close()

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')

cur = con.cursor()

results = cur.execute('''
SELECT ice_cream.title AS translation,
       wrappers.title AS original
FROM ice_cream,
     wrappers 
WHERE 
    ice_cream.wrapper_id = wrappers.id
    AND
    original LIKE 'Б%';
''')

for result in results:
    print(result)

con.close()